<a href="https://colab.research.google.com/github/yohan223/KoFarms-v1.0/blob/main/KoFarms1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load datasets (replace with actual file paths)
ndvi_data = pd.read_csv("ndvi_data.csv")  # NDVI values over time
soil_moisture_data = pd.read_csv("soil_moisture.csv")  # Soil moisture readings
soil_temp_data = pd.read_csv("soil_temperature.csv")  # Soil temperature readings
crop_yield_data = pd.read_csv("crop_yield.csv")  # Crop yield per season

# Preview datasets
print(ndvi_data.head())
print(soil_moisture_data.head())
print(soil_temp_data.head())
print(crop_yield_data.head())

# Merge datasets on common columns like 'date' or 'location'
merged_df = ndvi_data.merge(soil_moisture_data, on=['date', 'location'], how='inner')
merged_df = merged_df.merge(soil_temp_data, on=['date', 'location'], how='inner')
merged_df = merged_df.merge(crop_yield_data, on=['date', 'location'], how='inner')

# Handle missing values
merged_df.fillna(method='ffill', inplace=True)  # Forward fill missing values

# Convert date column to datetime format
merged_df['date'] = pd.to_datetime(merged_df['date'])

# Exploratory Data Analysis (EDA)
plt.figure(figsize=(12, 6))
sns.heatmap(merged_df.corr(), annot=True, cmap="coolwarm")
plt.title("Correlation Heatmap of Variables")
plt.show()

# Scatter plots to analyze relationships
sns.pairplot(merged_df, vars=['ndvi', 'soil_moisture', 'soil_temperature', 'crop_yield'])
plt.show()

# Time series visualization
plt.figure(figsize=(12, 6))
plt.plot(merged_df['date'], merged_df['ndvi'], label='NDVI', color='green')
plt.plot(merged_df['date'], merged_df['crop_yield'], label='Crop Yield', color='blue')
plt.xlabel('Date')
plt.ylabel('Values')
plt.title('NDVI vs Crop Yield Over Time')
plt.legend()
plt.show()

# Save cleaned data
merged_df.to_csv("cleaned_data.csv", index=False)

# Model Training
X = merged_df[['ndvi', 'soil_moisture', 'soil_temperature']]
y = merged_df['crop_yield']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Model Evaluation
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'R-squared Score: {r2}')

print("Data preprocessing and model training complete. Cleaned dataset saved as 'cleaned_data.csv'.")


FileNotFoundError: [Errno 2] No such file or directory: 'ndvi_data.csv'

In [7]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.5 MB/s eta 0:00:00


In [11]:
import ee
import pandas as pd
import streamlit as st

# Ensure authentication
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project='ee-johnslick1999')

# Load datasets
smap = ee.ImageCollection("NASA_USDA/HSL/SMAP_soil_moisture")
soil_temp = ee.ImageCollection("OpenLandMap/SOL/SOILTEMP")
soil_texture = ee.ImageCollection("OpenLandMap/SOL/SOILTEXTURE")
ndvi = ee.ImageCollection("COPERNICUS/S2").filterDate(str(start_date), str(end_date)).filterBounds(roi)

# Compute mean values
smap_mean = smap.filterDate(str(start_date), str(end_date)).mean().clip(roi)
soil_temp_mean = soil_temp.mean().clip(roi)
soil_texture_mean = soil_texture.mean().clip(roi)
ndvi_mean = ndvi.mean().clip(roi)

# Extract values as dictionaries
smap_data = smap_mean.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi, scale=1000).getInfo()
soil_temp_data = soil_temp_mean.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi, scale=1000).getInfo()
soil_texture_data = soil_texture_mean.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi, scale=1000).getInfo()
ndvi_data = ndvi_mean.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi, scale=10).getInfo()

# Create DataFrame
results = {"Parameter": ["Soil Moisture", "Soil Temperature", "Soil Texture", "NDVI"],
           "Value": [smap_data, soil_temp_data, soil_texture_data, ndvi_data]}
df = pd.DataFrame(results)

# Display results
st.write("### Analysis Results")
st.dataframe(df)

# Save CSV
csv_file = "crop_analysis_results.csv"
df.to_csv(csv_file, index=False)
st.download_button(label="Download CSV", data=df.to_csv().encode(), file_name=csv_file)
soil_temp = ee.ImageCollection("OpenLandMap/SOL/SOILTEMP")
soil_texture = ee.ImageCollection("OpenLandMap/SOL/SOILTEXTURE")
ndvi = ee.ImageCollection("COPERNICUS/S2").filterDate(str(start_date), str(end_date)).filterBounds(roi)

# Compute mean values
smap_mean = smap.filterDate(str(start_date), str(end_date)).mean().clip(roi)
soil_temp_mean = soil_temp.mean().clip(roi)
soil_texture_mean = soil_texture.mean().clip(roi)
ndvi_mean = ndvi.mean().clip(roi)

# Extract values as dictionaries
smap_data = smap_mean.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi, scale=1000).getInfo()
soil_temp_data = soil_temp_mean.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi, scale=1000).getInfo()
soil_texture_data = soil_texture_mean.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi, scale=1000).getInfo()
ndvi_data = ndvi_mean.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi, scale=10).getInfo()

# Create DataFrame
results = {"Parameter": ["Soil Moisture", "Soil Temperature", "Soil Texture", "NDVI"],
           "Value": [smap_data, soil_temp_data, soil_texture_data, ndvi_data]}
df = pd.DataFrame(results)

# Display results
st.write("### Analysis Results")
st.dataframe(df)

# Ensure authentication
ee.Authenticate()
##try:
   ##ee.Initialize()
#except Exception as e:
    #ee.Authenticate()
    #ee.Initialize()##

# Load datasets
smap = ee.ImageCollection("NASA_USDA/HSL/SMAP_soil_moisture")
soil_temp = ee.ImageCollection("OpenLandMap/SOL/SOILTEMP")
soil_texture = ee.ImageCollection("OpenLandMap/SOL/SOILTEXTURE")
ndvi = ee.ImageCollection("COPERNICUS/S2").filterDate(str(start_date), str(end_date)).filterBounds(roi)

# Compute mean values
smap_mean = smap.filterDate(str(start_date), str(end_date)).mean().clip(roi)
soil_temp_mean = soil_temp.mean().clip(roi)
soil_texture_mean = soil_texture.mean().clip(roi)
ndvi_mean = ndvi.mean().clip(roi)

# Extract values as dictionaries
smap_data = smap_mean.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi, scale=1000).getInfo()
soil_temp_data = soil_temp_mean.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi, scale=1000).getInfo()
soil_texture_data = soil_texture_mean.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi, scale=1000).getInfo()
ndvi_data = ndvi_mean.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi, scale=10).getInfo()

# Create DataFrame
results = {"Parameter": ["Soil Moisture", "Soil Temperature", "Soil Texture", "NDVI"],
           "Value": [smap_data, soil_temp_data, soil_texture_data, ndvi_data]}
df = pd.DataFrame(results)

# Display results
st.write("### Analysis Results")
st.dataframe(df)

# Save CSV
csv_file = "crop_analysis_results.csv"
df.to_csv(csv_file, index=False)
st.download_button(label="Download CSV", data=df.to_csv().encode(), file_name=csv_file)

/usr/local/lib/python3.11/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for NASA_USDA/HSL/SMAP_soil_moisture! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/NASA_USDA_HSL_SMAP_soil_moisture

  warnings.warn(warning, category=DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for COPERNICUS/S2! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

  warnings.warn(warning, category=DeprecationWarning)


NameError: name 'start_date' is not defined

In [12]:
!pip install geemap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 42.1 MB/s eta 0:00:00


In [19]:
import ee
import pandas as pd
import streamlit as st
import geemap.foliumap as geemap  # Import geemap for interactive mapping

# Ensure authentication
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project='ee-johnslick1999')

# Create a map instance
Map = geemap.Map()

# Display the map in Streamlit
Map_streamlit = st.components.v1.html(
    Map._repr_html_(), width=700, height=500
)

# Get ROI from user interaction
roi = Map.draw_last_feature

# Check if ROI is drawn
if roi:
    # Get start and end dates from user input
    start_date = st.date_input("Start Date", value=pd.to_datetime("2023-01-01"))
    end_date = st.date_input("End Date", value=pd.to_datetime("2023-12-31"))

    # Load datasets
    smap = ee.ImageCollection("NASA_USDA/HSL/SMAP_soil_moisture")
    soil_temp = ee.ImageCollection("OpenLandMap/SOL/SOILTEMP")
    soil_texture = ee.ImageCollection("OpenLandMap/SOL/SOILTEXTURE")
    ndvi = ee.ImageCollection("COPERNICUS/S2").filterDate(str(start_date), str(end_date)).filterBounds(roi.geometry())

    # ... (rest of your code, using roi.geometry() for calculations) ...

else:
    st.write("Please draw a region of interest (ROI) on the map.")

2025-02-16 09:34:38.066 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-16 09:34:38.067 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-16 09:34:38.073 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-16 09:34:38.074 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-16 09:34:38.078 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-16 09:34:38.079 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [2]:
import ee
ee.Authenticate()


Successfully saved authorization token.
